In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [5]:
# # CSV 파일 가져오기
# from google.colab import files
# uploaded = files.upload()

csv_df = pd.read_csv('dataset.csv')
csv_df

# plt.plot(csv_df,color='red')
# plt.title("주식 차트")
# plt.xlabel('Days')
# plt.ylabel('Price')
# plt.show

,Unnamed: 0,Date,환율,금리,y,ROE,영업이익률,부채비율,EPS,순이익률,BPS,PER,PBR,종가
0,0,2013-01-01,1062.500000,2.75,31520.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175,27440.0
1,1,2013-01-02,1062.500000,2.75,31520.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175,27440.0
2,2,2013-01-03,1062.599976,2.75,30860.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175,27440.0
3,3,2013-01-04,1061.900024,2.75,30500.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175,27440.0
4,4,2013-01-07,1033.099976,2.75,30400.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175,27440.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,3647,2022-12-18,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780,55300.0
3648,3648,2022-12-24,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780,55300.0
3649,3649,2022-12-25,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780,55300.0
3650,3650,2022-12-31,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780,55300.0


In [6]:
# 종가, 금리, 환율 등의 데이터만 모두 불러오고, 나머지 재무제표 데이터는 그 해 년도를 그대로 적용하여 사용

In [7]:
csv_df = csv_df.drop(['Unnamed: 0', '종가'], axis=1)
csv_df

,Date,환율,금리,y,ROE,영업이익률,부채비율,EPS,순이익률,BPS,PER,PBR
0,2013-01-01,1062.500000,2.75,31520.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175
1,2013-01-02,1062.500000,2.75,31520.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175
2,2013-01-03,1062.599976,2.75,30860.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175
3,2013-01-04,1061.900024,2.75,30500.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175
4,2013-01-07,1033.099976,2.75,30400.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175
...,...,...,...,...,...,...,...,...,...,...,...,...
3647,2022-12-18,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780
3648,2022-12-24,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780
3649,2022-12-25,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780
3650,2022-12-31,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780


In [8]:
csv_df = csv_df.rename(columns={'y':"종가"})
csv_df

,Date,환율,금리,종가,ROE,영업이익률,부채비율,EPS,순이익률,BPS,PER,PBR
0,2013-01-01,1062.500000,2.75,31520.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175
1,2013-01-02,1062.500000,2.75,31520.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175
2,2013-01-03,1062.599976,2.75,30860.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175
3,2013-01-04,1061.900024,2.75,30500.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175
4,2013-01-07,1033.099976,2.75,30400.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175
...,...,...,...,...,...,...,...,...,...,...,...,...
3647,2022-12-18,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780
3648,2022-12-24,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780
3649,2022-12-25,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780
3650,2022-12-31,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780


In [9]:
csv_df.dropna()

,Date,환율,금리,종가,ROE,영업이익률,부채비율,EPS,순이익률,BPS,PER,PBR
0,2013-01-01,1062.500000,2.75,31520.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175
1,2013-01-02,1062.500000,2.75,31520.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175
2,2013-01-03,1062.599976,2.75,30860.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175
3,2013-01-04,1061.900024,2.75,30500.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175
4,2013-01-07,1033.099976,2.75,30400.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175
...,...,...,...,...,...,...,...,...,...,...,...,...
3647,2022-12-18,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780
3648,2022-12-24,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780
3649,2022-12-25,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780
3650,2022-12-31,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780


In [10]:
#날짜 빼고 정규화

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scale_cols = ['환율','금리','종가','ROE','영업이익률','부채비율','EPS','순이익률','BPS','PER','PBR']

#정규화 수행
scaled_df = scaler.fit_transform(csv_df[scale_cols]) #정규화된 데이터는 넘파이 형태
scaled_df = pd.DataFrame(scaled_df, columns = scale_cols) #Pandas DataFrame 형태로 변경

#테스트
scaled_df

,환율,금리,종가,ROE,영업이익률,부채비율,EPS,순이익률,BPS,PER,PBR
0,0.141107,0.818182,0.146138,1.000000,0.333160,1.0,1.000000,0.433271,0.816624,0.000000,0.003474
1,0.141107,0.818182,0.146138,1.000000,0.333160,1.0,1.000000,0.433271,0.816624,0.000000,0.003474
2,0.141332,0.818182,0.136664,1.000000,0.333160,1.0,1.000000,0.433271,0.816624,0.000000,0.003474
3,0.139756,0.818182,0.131496,1.000000,0.333160,1.0,1.000000,0.433271,0.816624,0.000000,0.003474
4,0.074910,0.818182,0.130060,1.000000,0.333160,1.0,1.000000,0.433271,0.816624,0.000000,0.003474
...,...,...,...,...,...,...,...,...,...,...,...
3647,0.586382,1.000000,0.318117,0.666542,0.190005,0.0,0.025159,1.000000,0.021155,0.321004,0.528596
3648,0.586382,1.000000,0.318117,0.666542,0.190005,0.0,0.025159,1.000000,0.021155,0.321004,0.528596
3649,0.586382,1.000000,0.318117,0.666542,0.190005,0.0,0.025159,1.000000,0.021155,0.321004,0.528596
3650,0.586382,1.000000,0.318117,0.666542,0.190005,0.0,0.025159,1.000000,0.021155,0.321004,0.528596


In [11]:
scaled_csv_df = pd.concat([csv_df['Date'], scaled_df], axis=1)
scaled_csv_df = scaled_csv_df.set_index('Date')
scaled_csv_df

,환율,금리,종가,ROE,영업이익률,부채비율,EPS,순이익률,BPS,PER,PBR
Date,,,,,,,,,,,
2013-01-01,0.141107,0.818182,0.146138,1.000000,0.333160,1.0,1.000000,0.433271,0.816624,0.000000,0.003474
2013-01-02,0.141107,0.818182,0.146138,1.000000,0.333160,1.0,1.000000,0.433271,0.816624,0.000000,0.003474
2013-01-03,0.141332,0.818182,0.136664,1.000000,0.333160,1.0,1.000000,0.433271,0.816624,0.000000,0.003474
2013-01-04,0.139756,0.818182,0.131496,1.000000,0.333160,1.0,1.000000,0.433271,0.816624,0.000000,0.003474
2013-01-07,0.074910,0.818182,0.130060,1.000000,0.333160,1.0,1.000000,0.433271,0.816624,0.000000,0.003474
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-18,0.586382,1.000000,0.318117,0.666542,0.190005,0.0,0.025159,1.000000,0.021155,0.321004,0.528596
2022-12-24,0.586382,1.000000,0.318117,0.666542,0.190005,0.0,0.025159,1.000000,0.021155,0.321004,0.528596
2022-12-25,0.586382,1.000000,0.318117,0.666542,0.190005,0.0,0.025159,1.000000,0.021155,0.321004,0.528596


In [12]:
#입력데이터, 정답데이터 정의
feature_cols = ['환율','금리','종가','ROE','영업이익률','부채비율','EPS','순이익률','BPS','PER','PBR'] 
label_cols = ['종가'] 

#입력, 정답데이터 프레임
feature_df = scaled_csv_df[feature_cols]
label_df = scaled_csv_df[label_cols]

print(feature_df)
print(label_df)

feature_np = feature_df.to_numpy()
label_np = label_df.to_numpy()

                  환율        금리        종가       ROE     영업이익률  부채비율       EPS   
Date                                                                           
2013-01-01  0.141107  0.818182  0.146138  1.000000  0.333160   1.0  1.000000  \
2013-01-02  0.141107  0.818182  0.146138  1.000000  0.333160   1.0  1.000000   
2013-01-03  0.141332  0.818182  0.136664  1.000000  0.333160   1.0  1.000000   
2013-01-04  0.139756  0.818182  0.131496  1.000000  0.333160   1.0  1.000000   
2013-01-07  0.074910  0.818182  0.130060  1.000000  0.333160   1.0  1.000000   
...              ...       ...       ...       ...       ...   ...       ...   
2022-12-18  0.586382  1.000000  0.318117  0.666542  0.190005   0.0  0.025159   
2022-12-24  0.586382  1.000000  0.318117  0.666542  0.190005   0.0  0.025159   
2022-12-25  0.586382  1.000000  0.318117  0.666542  0.190005   0.0  0.025159   
2022-12-31  0.586382  1.000000  0.318117  0.666542  0.190005   0.0  0.025159   
2023-01-01  0.586382  1.000000  0.318117

In [13]:
window_size = 40  #40개의 입력데이터를 이용해 바로 다음 값에 오는 Close 값을 예측

def make_sequence_dataset(feature, label, window_size):

  feature_list = []   #생성될 feature list
  label_list = []     #생성될 label list

  for i in range(len(feature)-window_size):

    feature_list.append(feature[i:i+window_size])
    label_list.append(label[i+window_size])

  return np.array(feature_list), np.array(label_list)


X, y = make_sequence_dataset(feature_np, label_np, window_size)

print(X.shape, y.shape)

(3612, 40, 11) (3612, 1)


In [14]:
split = int(len(feature_df)*0.7) #테스트 데이터로 분리 -> train:test = 7:3

X_train = X[0:split]
y_train = y[0:split]

X_test = X[split:]
y_test = y[split:]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(2556, 40, 11) (2556, 1)
(1056, 40, 11) (1056, 1)


In [15]:
#LSTM 모델 구축
model = Sequential()

#LSTM layer
model.add(LSTM(128,
               activation = 'sigmoid',
               input_shape = X_train[0].shape))   #input_shape = (2,8)
#출력층
model.add(Dense(11, activation = 'sigmoid'))
model.add(Dense(1)) #출력층
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               71680     
                                                                 
 dense (Dense)               (None, 11)                1419      
                                                                 
 dense_1 (Dense)             (None, 1)                 12        
                                                                 
Total params: 73,111
Trainable params: 73,111
Non-trainable params: 0
_________________________________________________________________


In [16]:
# 모델 컴파일
model.compile(loss='mse', optimizer='adam', metrics=['mae'])  # 옵티마이저와 손실 함수, 평가지표 설정

# 모델 학습
model.fit(X_train, y_train, epochs=100, batch_size=32)

# 주가 예측
predictions = model.predict(X_test)

Epoch 1/100
80/80 [==============================] - 2s 13ms/step - loss: 0.1503 - mae: 0.2872
Epoch 2/100
80/80 [==============================] - 1s 14ms/step - loss: 0.0489 - mae: 0.1785
Epoch 3/100
80/80 [==============================] - 1s 15ms/step - loss: 0.0313 - mae: 0.1329
Epoch 4/100
80/80 [==============================] - 1s 14ms/step - loss: 0.0215 - mae: 0.1092
Epoch 5/100
80/80 [==============================] - 1s 16ms/step - loss: 0.0172 - mae: 0.1024
Epoch 6/100
80/80 [==============================] - 1s 13ms/step - loss: 0.0133 - mae: 0.0911
Epoch 7/100
80/80 [==============================] - 1s 14ms/step - loss: 0.0080 - mae: 0.0700
Epoch 8/100
80/80 [==============================] - 1s 14ms/step - loss: 0.0038 - mae: 0.0459
Epoch 9/100
80/80 [==============================] - 1s 14ms/step - loss: 0.0018 - mae: 0.0322
Epoch 10/100
80/80 [==============================] - 1s 14ms/step - loss: 0.0014 - mae: 0.0277
Epoch 11/100
80/80 [=============================

80/80 [==============================] - 1s 13ms/step - loss: 2.1242e-04 - mae: 0.0108
Epoch 84/100
80/80 [==============================] - 1s 13ms/step - loss: 2.8331e-04 - mae: 0.0130
Epoch 85/100
80/80 [==============================] - 1s 13ms/step - loss: 2.1837e-04 - mae: 0.0111
Epoch 86/100
80/80 [==============================] - 1s 13ms/step - loss: 1.9620e-04 - mae: 0.0103
Epoch 87/100
80/80 [==============================] - 1s 13ms/step - loss: 2.0876e-04 - mae: 0.0108
Epoch 88/100
80/80 [==============================] - 1s 13ms/step - loss: 2.1346e-04 - mae: 0.0110
Epoch 89/100
80/80 [==============================] - 1s 13ms/step - loss: 1.9471e-04 - mae: 0.0104
Epoch 90/100
80/80 [==============================] - 1s 13ms/step - loss: 1.8887e-04 - mae: 0.0101
Epoch 91/100
80/80 [==============================] - 1s 13ms/step - loss: 1.8925e-04 - mae: 0.0100
Epoch 92/100
80/80 [==============================] - 1s 14ms/step - loss: 1.9000e-04 - mae: 0.0103
Epoch 93/100


In [14]:
print(predictions)

[[0.55505884]
 [0.5475597 ]
 [0.54097736]
 ...
 [0.2928343 ]
 [0.29283765]
 [0.29284057]]


In [65]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# MAE, RMSE, R-Squared 계산
mae = mean_absolute_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

# 결과 출력
print("MAE: {:.5f}".format(mae))
print("RMSE: {:.5f}".format(rmse))
print("R-Squared: {:.5f}".format(r2))  


MAE: 0.02083
RMSE: 0.02660
R-Squared: 0.96941


In [66]:
close_min = csv_df['종가'].min()
close_max = csv_df['종가'].max()

print(csv_df['종가'].min())
print(csv_df['종가'].max())

21340.0
91000.0


In [75]:
#실제값 변환!
scaler2 = MinMaxScaler()
scaled_df2 = scaler2.fit_transform(csv_df[['종가']])


# MinMaxScaler에 정규화에 사용한 최솟값과 최댓값을 설정 -----------> 문제점이 있다. 아래에서 확인...
scaler2.data_min_ = close_min  # 정규화에 사용한 최솟값
scaler2.data_max_ = close_max  # 정규화에 사용한 최댓값

# 예측한 출력값을 실제값으로 역변환
original_pred_values = scaler2.inverse_transform(predictions)

# 역변환된 예측값 출력
print(original_pred_values)

[[59619.723]
 [59106.15 ]
 [58639.85 ]
 ...
 [41910.363]
 [41909.574]
 [41908.91 ]]


In [86]:
original_label_df = csv_df[label_cols]
original_y_train = original_label_df[0:split]
original_y_test = original_label_df[split:]
original_y_test = original_y_test[window_size:] #윈도우 사이즈 만큼의 데이터도 빼야 했다...


#실제값과 비교
print(original_y_test)

           종가
2596  58600.0
2597  58000.0
2598  59100.0
2599  58100.0
2600  57900.0
...       ...
3647  43500.0
3648  43500.0
3649  43500.0
3650  43500.0
3651  43500.0

[1056 rows x 1 columns]


In [83]:
# MAE, RMSE, R-Squared 계산
mae = mean_absolute_error(original_y_test, original_pred_values)
rmse = mean_squared_error(original_y_test, original_pred_values, squared=False)
r2 = r2_score(original_y_test, original_pred_values)

# 결과 출력
print("MAE: {:.5f}".format(mae))
print("RMSE: {:.5f}".format(rmse))
print("R-Squared: {:.5f}".format(r2))  


MAE: 1450.97045
RMSE: 1853.22443
R-Squared: 0.96941


**문제점:**

실제 값으로 변환하기 위해, 출력 데이터의 정규화 이전 값의 최솟값과 최댓값, 즉 종가데이터의 최솟값과 최댓값 기반으로 범위를 설정하였다...

내 생각이 맞다면 지금 테스트 상에서는 테스트셋의 예측값이 최솟값과 최댓값 범위 사이에 있어 정확도가 맞아 떨어진 경우같음...

만약 예측 값이 기존 종가데이터의 최솟값 이하이거나, 최댓값 이상이라면...?
아무리 실제랑 차이가 나더라도 종가 데이터셋의 최솟값과 최댓값 사이의 범위안으로 가격이 한정된다.
현 주가가 기존 데이터셋의 최댓값에 도달하고 그 이상 뛰어넘을 때, 아무리 가격이 올라가더라도 예측값이 최댓값에 머문다는 뜻...

<챗gpt의 의견>

```
예측한 값이 정규화된 출력 데이터의 최소값 이하나 최대값 이상이 될 수 있다는 점에 주의해야 합니다. 정규화된 데이터의 범위는 일반적으로 0과 1 사이로 제한되기 때문에, 예측한 값도 해당 범위 내에 존재할 것으로 예상됩니다.

만약 예측한 값이 정규화된 범위를 벗어날 경우, 이는 역변환이 수행되지 않은 상태에서의 예측이므로 역변환 후의 실제값이 아닐 수 있습니다. 이러한 경우에는 역변환 이후에도 정규화된 범위 내로 값을 조정해야 합니다.

예를 들어, 예측한 값을 역변환한 후에도 최소값 이하나 최대값 이상인 경우에는 해당 값을 최소값 또는 최대값으로 강제로 조정하는 등의 후처리 작업을 수행해야 합니다. 이렇게 함으로써 예측값이 데이터의 유효 범위 내에 유지될 수 있습니다.

따라서, 예측한 값을 역변환할 때 정규화된 데이터의 최소값과 최대값을 기준으로 범위를 확인하고, 이를 넘어가는 값에 대해서는 추가적인 조치를 취해야 합니다.```